In [10]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Chuẩn bị dữ liệu

In [11]:
def train_test_animals():
    from tensorflow.keras.preprocessing.image import ImageDataGenerator

    # Số lượng ảnh mỗi lớp cho tập huấn luyện và tập kiểm tra
    images_per_class_train = 750
    images_per_class_test = 250

    # Tạo ImageDataGenerator
    train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.25)

    # Tạo tập huấn luyện
    train_generator = train_datagen.flow_from_directory(
        'D:/Code/python/mat3533/practice08/data/CNN_MultiClass_data/animals',
        target_size=(150, 150),
        batch_size=images_per_class_train,  # Lấy hết ảnh trong mỗi lớp cho tập huấn luyện
        class_mode='categorical',
        subset='training',
        shuffle=True  # Xáo trộn dữ liệu
    )

    # Tạo tập kiểm tra
    test_generator = train_datagen.flow_from_directory(
        'D:/Code/python/mat3533/practice08/data/CNN_MultiClass_data/validation',
        target_size=(150, 150),
        batch_size=images_per_class_test,  # Lấy hết ảnh trong mỗi lớp cho tập kiểm tra
        class_mode='categorical',
        subset='validation',
        shuffle=True  # Xáo trộn dữ liệu
    )

    # Lấy ảnh và nhãn từ tập dữ liệu
    train_images, labels_train = next(train_generator)
    test_images, labels_test = next(test_generator)

    return train_images, test_images, labels_train, labels_test

X_train, X_test, y_train, y_test = train_test_animals()


Found 2250 images belonging to 3 classes.
Found 750 images belonging to 3 classes.


Đưa ảnh về dạng vector

In [12]:
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

y_train = np.argmax(y_train, axis=1)
y_test = np.argmax(y_test, axis=1)

Giảm dữ liệu xuống 225 chiều

In [13]:
from sklearn.decomposition import PCA

n_components = 225

pca = PCA(n_components=n_components)
pca.fit(X_train)

X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

Khởi tạo mô hình ANN

In [14]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten

model = Sequential()

model.add(Flatten(input_shape = (225,)))
model.add(Dense(128,activation = 'relu'))
model.add(Dense(64,activation = 'relu'))
model.add(Dense(32,activation = 'relu'))
model.add(Dense(10,activation = 'softmax')) 

model.compile(loss = 'sparse_categorical_crossentropy', 
              optimizer = 'Adam', 
              metrics = ['accuracy'])

model.fit(X_train_pca,y_train,epochs= 4 , validation_split = .2)

c:\Users\Admin\anaconda3\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/4
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.3182 - loss: 2.2600 - val_accuracy: 0.4933 - val_loss: 1.0918
Epoch 2/4
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6832 - loss: 0.7739 - val_accuracy: 0.5467 - val_loss: 0.9340
Epoch 3/4
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8513 - loss: 0.4420 - val_accuracy: 0.5667 - val_loss: 0.9224
Epoch 4/4
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9108 - loss: 0.2996 - val_accuracy: 0.6067 - val_loss: 0.9780


Chạy dự đoán dữ liệu test

In [18]:
y_pred = model.predict(X_test_pca)
y_pred = y_pred.argmax(axis = 1)

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


Tính độ chính xác

In [19]:
accuracy = accuracy_score(y_pred, y_pred)
print("Accuracy:", accuracy)

Accuracy: 1.0
